In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import os

Warning in <TInterpreter::ReadRootmapFile>: class  HepMC::FourVector found in libSimDataFormatsGeneratorProducts.so  is already in libHepMC3rootIO.so 


Welcome to JupyROOT 6.28/00


/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  r

In [2]:
#import helper modules for muon scale factor computation
sys.path.append("/uscms/home/amalbert/nobackup/CMSSW_14_1_0_pre4/src/RazorCommon/Tools/bin")
import importlib
import getMuonScaleFactor

In [3]:
ak.behavior.update(candidate.behavior)

def getLZDF(f,nEvents=-1,version="new"): #lazy dataframe with events that have cluster matched to probe muon
    events_raw = uproot.open(f)['MuonSystem']
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    #events = events[start:stop]
    return events

In [4]:
#paths
MC_paths = {"2022":"/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters/MC_Summer22/DYto2Mu_MLL-50to120_keepMDSHits_Merged/DYto2Mu_MLL-50to120_keepMDSHits_7980pb_weighted.root",
            "2022EE":"/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters_fixed/MC_Summer22EE/DYto2Mu_MLL-50to120_keepMDSHits_Merged/DYto2Mu_MLL-50to120_keepMDSHits.root", #couldn't normalize ntuple, will do manually
            "2023":"/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters_fixed/MC_Summer23/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120.root",
            "2023BPix":"/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters_fixed/MC_Summer23BPix/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120.root"}

data_path_lists = {"2022":["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2022_Merged/Muon_Run2022C_PromptReco-v1_goodLumi.root",
                          "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2022_Merged/Muon_Run2022D_PromptReco-v1_goodLumi.root"],
                 
                   "2022EE":["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2022_Merged/Muon_Run2022E_PromptReco-v1_goodLumi.root",
                          "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2022_Merged/Muon_Run2022F_PromptReco-v1_goodLumi.root",
                            "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2022_Merged/Muon_Run2022G_PromptReco-v1_goodLumi.root"],
                   
                    "2023":["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023B_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023B_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023C_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023C_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023C_PromptReco-v2_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023C_PromptReco-v2_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023C_PromptReco-v3_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023C_PromptReco-v3_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023C_PromptReco-v4_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023C_PromptReco-v4_goodLumi.root"],
                  
                      "2023BPix":["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023D_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023D_PromptReco-v1_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon0_Run2023D_PromptReco-v2_goodLumi.root",
                     "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters_fixed/2023_Merged/Muon1_Run2023D_PromptReco-v2_goodLumi.root"]}

In [5]:
#events_data = getLZDF(data_path)
events_MC_full_dict= {}
for campaign, MC_path in MC_paths.items():
    print(campaign)
    events_MC_full = getLZDF("root://cmseos.fnal.gov/"+MC_path)
    events_MC_full = events_MC_full[np.logical_and(events_MC_full.Flag_all, events_MC_full.Flag_ecalBadCalibFilter)]
    events_MC_full = events_MC_full[events_MC_full.jetVeto]
    events_MC_full_dict[campaign] = events_MC_full

2022
2022EE
2023
2023BPix


In [6]:
events_data_full_dict= {}
for campaign, data_path_list in data_path_lists.items():
    print(campaign)
    data_events = [getLZDF("root://cmseos.fnal.gov/"+data_path) for data_path in data_path_list]
    events_data_full = ak.concatenate(data_events, axis=0)
    events_data_full = events_data_full[np.logical_and(events_data_full.ZMass>50, events_data_full.ZMass<120)]
    events_data_full = events_data_full[np.logical_and(events_data_full.Flag_all, events_data_full.Flag_ecalBadCalibFilter)]
    events_data_full = events_data_full[events_data_full.jetVeto]
    events_data_full_dict[campaign] = events_data_full

2022
2022EE
2023
2023BPix


In [7]:
branch_names = ["weight", "pileupWeight", "ZMass"]

In [8]:
MC_lumiWeights = {"2022":1,"2022EE":2.0492258,"2023":7.1269202,"2023BPix":7.4827265}

In [9]:
def getBranches(LZDF, campagin, isMC=False):
    new_df = ak.zip({field: np.repeat(LZDF[field], LZDF["numTag"]) for field in branch_names})
#     for branch in branch_names:
#         if (not isMC) and (branch in ["weight", "pileupWeight"]):
#             continue
#         new_df = ak.with_field(new_df, np.repeat(LZDF[branch],LZDF["numTag"]), branch)
    print("finished event level variables")
    
    lepTag_array = LZDF["lepTag"]
    lepTag_array = np.repeat(lepTag_array,np.array(LZDF["numTag"]), axis=0)
    arr = np.zeros(np.shape(lepTag_array)[0], dtype=bool)
    arr[::2] = 1  # Set even indices to 1
    mask = ak.all(lepTag_array, axis=1)
    mask_T_F = np.logical_and(arr, mask)
    mask_F_T = np.logical_and(np.logical_not(arr), mask)
    lepTag_array = np.array(lepTag_array)
    lepTag_array.reshape((np.array(mask_T_F).shape[0],2))
    lepTag_array[mask_T_F] = [True, False]
    lepTag_array[mask_F_T] = [False, True]
    
    row_indices = np.arange(lepTag_array.shape[0], dtype=int)
    column_indices_probe =np.where(lepTag_array==False)[1]
    column_indices_tag =np.where(lepTag_array==True)[1]
    
    
    if isMC:
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        
        MC_SF_LooseID = np.repeat(MC_SF_LooseID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_LooseISO = np.repeat(np.array(MC_SF_LooseISO),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_TightID = np.repeat(MC_SF_TightID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_TightISO = np.repeat(MC_SF_TightISO,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_HLT = np.repeat(MC_SF_HLT,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        
        MC_Weight_Total = MC_lumiWeights[campagin]*MC_SF_LooseID*MC_SF_LooseISO*MC_SF_TightID*MC_SF_TightISO*MC_SF_HLT
        new_df = ak.with_field(new_df, MC_Weight_Total, "weight_total")
    
    print("at muon variables")
    #load pT, eta, and phi for tag and probe muons
    probe_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    
    tag_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    
    new_df = ak.with_field(new_df, probe_pT, "probe_pT")
    new_df = ak.with_field(new_df, probe_eta, "probe_eta")
    new_df = ak.with_field(new_df, probe_phi, "probe_phi")
    
    new_df = ak.with_field(new_df, tag_pT, "tag_pT")
    new_df = ak.with_field(new_df, tag_eta, "tag_eta")
    new_df = ak.with_field(new_df, tag_phi, "tag_phi")
    
    return new_df

In [ ]:
events_MC_dict = {}
events_data_dict = {}
for campaign in list(events_MC_full_dict.keys()):
    print(campaign)
    print("MC")
    events_MC = getBranches(events_MC_full_dict[campaign], campaign, True)
    print("now data")
    events_data = getBranches(events_data_full_dict[campaign], campaign, False)
    events_MC_dict[campaign] = events_MC
    events_data_dict[campaign] = events_data

2022
MC
finished event level variables
at muon variables
now data
finished event level variables
at muon variables
2022EE
MC
finished event level variables
at muon variables
now data
finished event level variables
at muon variables
2023
MC
finished event level variables
at muon variables
now data
finished event level variables
at muon variables
2023BPix
MC
finished event level variables
at muon variables
now data
finished event level variables
at muon variables


In [ ]:
for campaign in list(events_MC_full_dict.keys()):
    print(campaign)
    events_data = events_data_dict[campaign]
    events_MC = events_MC_dict[campaign]
    print("# of data events: ", ak.size(events_data.ZMass))
    print('# of MC events: ', ak.sum(events_MC.weight_total))
    print("MC k scaling factor: ", ak.size(events_data.ZMass)/ak.sum(events_MC.weight_total))

In [ ]:
rt.gStyle.SetOptStat(0)
def make_ratio_plot(h_list_in, title = "", label = "", fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1]):
    h_list = []
    if in_tags == None:
        tag = []
    else:
        tag = in_tags
    for i, h in enumerate(h_list_in):
        h_list.append(h.Clone('h{}aux{}'.format(i, label)))
        if in_tags == None:
            tag.append(h.GetTitle())
    #print("tags: ", tag)
    c_out = rt.TCanvas("c_out_ratio"+label, "c_out_ratio"+label, 800, 800)
    pad1 = rt.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
    pad1.SetBottomMargin(0.03)
    pad1.SetLeftMargin(0.15)
    pad1.SetRightMargin(0.04)# pad2.SetGrid()
    if logy:
        pad1.SetLogy()

    pad1.Draw()
    pad1.cd()

    leg = rt.TLegend(0.5, 0.65, 0.9, 0.92)
    leg = rt.TLegend(0.7, 0.65, 0.9, 0.92)

    #leg = rt.TLegend(0.2, 0.7, 0.5, 0.9)
    # leg = rt.TLegend(0.7, 0.2, 0.9, 0.4)
    leg.SetBorderSize(0)
    leg.SetTextSize(0.045)
    leg.SetFillStyle(0)
    c_out.cd(1)

    scaled_h_list = []
    if scale:
        for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #h = h_unscaled.Scale(1/scales[i])
            #scaled_h_list.append(h_unscaled.Clone())
            h_unscaled.Scale(1/scales[i])
            scaled_h_list.append(h_unscaled)
    else:
        #for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #scaled_h_list.append(h)
        scaled_h_list = h_list
    for i, h in enumerate(scaled_h_list):
        h.GetXaxis().SetLabelSize(0)
        h.GetXaxis().SetTitle(label)
        h.GetYaxis().SetRangeUser(0, 1.1*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and not scale:
            h.GetYaxis().SetRangeUser(10e-2, 2*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and scale:
            h.GetYaxis().SetRangeUser(10e-4, 1)
        h.GetYaxis().SetTitleOffset(1.0)
        h.GetYaxis().SetTitleSize(0.06)
        h.GetYaxis().SetLabelSize(0.05)
        
        if scale:
            y_title = "Fraction of Events"
        else:
            y_title = "Events"
        
        h.GetYaxis().SetTitle()
        h.SetTitle(f"{title};adsf;{y_title}")
        #if ratio_index == 0:h.DrawCopy("hist")
        '''
        h.SetFillColor(h_list_in[i].GetLineColor())
        h.SetFillStyle(3002)
        #h.SetStats(1)
        h.SetLineColor(h_list_in[i].GetLineColor())
        h.SetLineWidth(2)
        h.SetMarkerColor(h_list_in[i].GetLineColor())
        h.SetMarkerSize(2)
        # if ratio_index == 0:
        #     # h.DrawCopy("hist")
        #     h.DrawCopy(draw_opt[i]+'same')
        # else:h.DrawCopy(draw_opt[i])
        #if ratio_index == 0 :h.DrawCopy(draw_opt[i]+"same")
        #h.DrawCopy("E2 HIST")
        '''
        if i==0:
            h.SetLineWidth(4)
            h.DrawCopy("hist")
            #h.SetFillStyle(0)
            h.SetFillColor(h_list_in[i].GetLineColor())
            h.SetFillStyle(3002)
            #h.SetStats(1)
            h.SetLineColor(h_list_in[i].GetLineColor())
            h.SetLineWidth(2)
            h.SetMarkerColor(h_list_in[i].GetLineColor())
            h.SetMarkerSize(2)
            h.DrawCopy(draw_opt[i] + "same")
            #h.SetFillStyle(0)
        else:
            h.SetLineWidth(2)
            h.DrawCopy(draw_opt[i] + "same")
        #else:h.DrawCopy(draw_opt[i])
        if len(text)>0:
            l = rt.TLatex()
            l.SetTextSize(0.045)
            if logy:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()/10, text)
            else:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()*0.8, text)
        #if i==1:
            #h.DrawCopy(draw_opt[i]+"same")
       #     h.Draw("E1 same")

        leg.AddEntry(h, tag[i], "lep")
    leg.Draw("same")

    c_out.cd()
    pad2 = rt.TPad("pad2", "pad2", 0, 0, 1, 0.3)
    pad2.SetTopMargin(0.03)
    pad2.SetBottomMargin(0.25)
    pad2.SetLeftMargin(0.15)
    pad2.SetRightMargin(0.04)# pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    band = scaled_h_list[ratio_index].Clone('h_band')
    for j in range(band.GetXaxis().GetNbins()):
        band.SetBinContent(j+1, 1.0)
        if h_list[ratio_index].GetBinContent(j+1) == 0:
            band.SetBinError(j+1, 0.0)
        else:
            band.SetBinError(j+1, scaled_h_list[ratio_index].GetBinError(j+1)/scaled_h_list[ratio_index].GetBinContent(j+1))
            #print(j, h_list_in[0].GetBinError(j+1)/h_list_in[0].GetBinContent(j+1))
    band.SetFillColor(scaled_h_list[ratio_index].GetLineColor())

    band.SetFillStyle(3002)
    band.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    #band.SetFillColorAlpha(0,0)
    #band.SetLineColor(0)
    
    band.GetYaxis().SetTitleOffset(0.5)
    band.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
    band.GetYaxis().SetTitleSize(0.11)
    band.GetYaxis().SetLabelSize(0.12)
    band.GetYaxis().SetNdivisions(506)
    band.GetXaxis().SetTitleOffset(0.95)
    band.GetXaxis().SetTitleSize(0.12)
    band.GetXaxis().SetLabelSize(0.12)
    band.GetXaxis().SetTickSize(0.07)
    
    band.SetYTitle('Ratio with {}'.format(tag[ratio_index]))
    band.SetXTitle(label)
    band.SetTitle("")
    band.DrawCopy('E2')
    ln = rt.TLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
    ln.SetLineWidth(3)
    ln.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    ln.DrawLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
     
    #print(ratio_index)
    for i, h in enumerate(scaled_h_list):
        if i == ratio_index:
            continue
        else:
            if fit:h.GetFunction("expo")
            h.Divide(scaled_h_list[ratio_index])
            # h.GetYaxis().SetTitleOffset(0.6)
            # h.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
            # h.GetYaxis().SetTitleSize(0.12)
            # h.GetYaxis().SetLabelSize(0.12)
            # h.GetYaxis().SetNdivisions(506)
            # h.GetXaxis().SetTitleOffset(0.95)
            # h.GetXaxis().SetTitleSize(0.12)
            # h.GetXaxis().SetLabelSize(0.12)
            # h.GetXaxis().SetTickSize(0.07)
            # h.SetYTitle('Ratio with {}'.format(tag[0]))
            # h.SetTitle("")
            #set relative error of ratio to be the relative error of data
            for j in range(h.GetXaxis().GetNbins()):
                if h_list[i].GetBinContent(j+1) == 0:
                    h.SetBinError(j+1, 0.0)
                else:
                    h.SetBinError(j+1, h_list_in[i].GetBinError(j+1)/h_list_in[i].GetBinContent(j+1)*h.GetBinContent(j+1))
            h.Draw('same'+draw_opt[i])
    
    pad2.Update()
    
    c_out.pad1 = pad1
    c_out.pad2 = pad2
    c_out.h_list = h_list
    c_out.leg = leg
    
    return c_out

In [ ]:
#helper function to build histograms

def makeHists(events_data, events_MC, branch, bins_tuple):
    print(f"on branch {branch}")
    
    nbins, lowBin, highBin = bins_tuple
    


    data_tree = events_data

    MC_tree = events_MC

    #initialize data and MC histograms
    data = rt.TH1F("Data", "Data", nbinsx=nbins, xlow = lowBin, xup=highBin)
    MC = rt.TH1F("MC", "MC", nbinsx=nbins, xlow = lowBin, xup=highBin)


    hist_info = {}
    
    #build data hist
    data_arr = np.array(data_tree[branch], dtype=np.float64)
    data_size = np.size(data_arr)
    data_weights = np.ones(data_size, dtype=np.float64)
    data.FillN(data_size, data_arr, data_weights)
    data.SetLineColor(rt.kBlack)
    data.SetFillStyle(0)

    #build MC hist
    MC_arr = np.array(MC_tree[branch], dtype=np.float64)
    MC_size = np.size(MC_arr)
    MC_weights = np.array(MC_tree["weight_total"])
    MC.FillN(MC_size, MC_arr, MC_weights)
    MC.SetLineColor(rt.kRed)
    MC.SetFillStyle(0)

    sumOfWeights = ak.sum(MC_tree["weight_total"])

    hist_info[branch] = {"MC_hist": MC, "data_hist": data, 
                                      "MC_weights": sumOfWeights, "data_weights": data_size}
        
    return hist_info
    

In [ ]:
#define dictionary with relevant plot info
plot_info = {"ZMass": {"filename_base":"ZMass", "title":"Dimuon Mass Distribution", 
                       "xlabel":"Dimuon Mass [GeV]", "masks":[],"bins":(80, 0, 150), "logy":False},
#              "puppiMet": {"filename_base":"puppiMet", "title":"PUPPI MET Distribution", 
#                        "xlabel":"PUPPI MET [GeV]", "masks":[],"bins":(60, -5, 100), "logy":False},
#             "puppiMetPhi": {"filename_base":"puppiMetPhi", "title":"PUPPI MET Phi Distribution", 
#                        "xlabel":"PUPPI MET Phi", "masks":[],"bins":(30, -4, 4), "logy":False},
            "probe_pT": {"filename_base":"probe_pT", "title":"pT of Muon Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(25, 0, 100), "logy":False},
            "tag_pT": {"filename_base":"tag_pT", "title":"pT of Muon Not Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(25, 0, 100), "logy":False},
            "probe_phi": {"filename_base":"probe_phi", "title":"Phi of Muon Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
            "tag_phi": {"filename_base":"tag_phi", "title":"Phi of Muon Not Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
            "probe_eta": {"filename_base":"probe_eta", "title":"Eta of Muon Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
            "tag_eta": {"filename_base":"tag_eta", "title":"Eta of Muon Not Matched to Cluster", 
                       "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
            }

In [ ]:

individual_plot_info_dict = {}
for campaign in list(events_MC_full_dict.keys()):
    print(campaign)
    individual_plot_info = {}
    for branch, info_dict in plot_info.items():
        filename_base = info_dict["filename_base"]
        hist_dict = makeHists(events_data, events_MC, branch, info_dict["bins"])
        for plot_hists, plot_hist_dict in hist_dict.items():
            individual_plot_info[plot_hists] = {"MC_hist": plot_hist_dict["MC_hist"], "data_hist": plot_hist_dict["data_hist"], 
                        "file_name": plot_hists, "title": info_dict["title"], "label": info_dict["xlabel"], 
                        "scales": [plot_hist_dict["MC_weights"], plot_hist_dict["data_weights"]], "logy": info_dict["logy"]}
        individual_plot_info_dict[campaign] = individual_plot_info


In [ ]:
for campaign in list(events_MC_full_dict.keys()):
    print(campaign)
    plot_output = f"{campaign}_Data_MC_Comp_finalzedSelections_testCode_noClusters"
    os.makedirs(plot_output, exist_ok=True)
    
    individual_plot_info = individual_plot_info_dict[campagin]
    for plot_type, plot_info_dict in individual_plot_info.items():
        print(plot_type)
        for boolScale in [True, False]:
            c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
            if boolScale:
                scaleString = "_normalized"
            else:
                scaleString=""
            os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
            c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+"_updatedTNP.png")
        